# Cropped image + CNN
Near 부분이 0 인 값만 Cropped 이미지로 대체하고 CNN 모델로 예측했지만 결과에 별 영향은 없어보입니다.
<br>
<br>
<br>
<br>
#RMSE 비교
### CNN
- rmse:21.0734

<br>

### Cropped image + CNN: 
-  rmse:21.0737

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import tensorflow as tf


path="/content/drive/MyDrive/petfinder-pawpularity-score"
train=pd.read_csv(path+'/train.csv')
test=pd.read_csv(path+'/test.csv')


In [13]:
train["file_path"] = train["Id"].apply(lambda x:path +"/train/"+ x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x:path +"/test/"+ x + ".jpg")

### Near 가 0 인 값 1372개 

In [ ]:
train[train["Near"]==0].sum

<bound method DataFrame.sum of                                     Id  ...                                          file_path
1     0009c66b9439883ba2750fb825e1d7db  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
5     001dd4f6fafb890610b1635f967ea081  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
7     0031d6a9ef7340f898c3e05f92c7bb04  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
14    00655425c10d4c082dd7eeb97fa4fb17  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
17    006fe962f5f7e2c5f527b2e27e28ed6d  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
...                                ...  ...                                                ...
9864  fe8502be7330dd591b7349492e594f3e  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
9888  ff1233d348cba9f95ba35679b35e26b6  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
9894  ff5aacc3eeb12cc9c029259d5171a4df  ...  /content/drive/MyDrive/petfinder-pawpularity-s...
9897  ff6591fb1b348

In [ ]:
train[train["Near"]==0].index   # Near 가 0 인 데이터는 1,5,7,14 ... index 값들임 

Int64Index([   1,    5,    7,   14,   17,   19,   26,   28,   37,   49,
            ...
            9831, 9842, 9849, 9856, 9863, 9864, 9888, 9894, 9897, 9903],
           dtype='int64', length=1372)

In [36]:
# 기존 데이터를 normal_image 에 cropped 데이터를 crop_image
crop_path="/content/drive/MyDrive/crop/"
train1=pd.read_csv(path+'/train.csv')
train1["crop_image"] = train["Id"].apply(lambda x:crop_path +x+ ".jpg")
train1["normal_image"] = train["Id"].apply(lambda x:path +"/train/"+ x + ".jpg")

In [39]:
# Near 가 0인 index를 뽑아서 그 index만 cropped 이미지로 바꾸고 New_image 데이터프레임에 새로 저장하였습니다.
idx=train[train['Near']==0].index
train1["New_image"]=train1["normal_image"]
for i in idx:
    train1["New_image"][idx]=train1["crop_image"][idx]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
len(idx)

1372

In [40]:
train1  # 1번째 값 cropped 으로 변경됨을 확인

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,crop_image,normal_image,New_image
index,,,,,,,,,,,,,,,,,
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,/content/drive/MyDrive/crop/0007de18844b0dbbb5...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,/content/drive/MyDrive/crop/0009c66b9439883ba2...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/crop/0009c66b9439883ba2...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,/content/drive/MyDrive/crop/0013fd999caf9a3efe...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,/content/drive/MyDrive/crop/0018df346ac9c1d841...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,/content/drive/MyDrive/crop/001dc955e10590d3ca...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15,/content/drive/MyDrive/crop/ffbfa0383c34dc513c...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70,/content/drive/MyDrive/crop/ffcc8532d76436fc79...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20,/content/drive/MyDrive/crop/ffdf2e8673a1da6fb8...,/content/drive/MyDrive/petfinder-pawpularity-s...,/content/drive/MyDrive/petfinder-pawpularity-s...


In [41]:
train1.iloc[17] #17번째 이미지가 cropped 이미지로 변경 되었음을 확인 

Id                                006fe962f5f7e2c5f527b2e27e28ed6d
Subject Focus                                                    0
Eyes                                                             1
Face                                                             1
Near                                                             0
Action                                                           0
Accessory                                                        0
Group                                                            0
Collage                                                          1
Human                                                            1
Occlusion                                                        1
Info                                                             0
Blur                                                             0
Pawpularity                                                     69
crop_image       /content/drive/MyDrive/crop/006fe962f5f7e2c5f

# CNN 모델 학습

In [42]:
image_size = 128
def preprocess(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_size, image_size))
    return image

In [43]:
x_train=[]
for i in train1['New_image']:
    x1=preprocess(i)
    x_train.append(x1)

In [44]:
test2=[]
for i in test['file_path']:
    x1=preprocess(i)
    test2.append(x1)
test2=np.array(test2)

In [45]:
x_train=np.array(x_train)
y_train=train['Pawpularity']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [46]:
inputs=keras.Input(shape=(128,128,3))
x=inputs
x=keras.layers.Conv2D(filters=4,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=8,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=16,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Flatten()(x)
x=keras.layers.Dense(128, activation = "relu")(x)
x=keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [47]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5
    )
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.1,
    patience=2, 
    min_lr=1e-9
    )
callbacks = [early_stop,reduce_lr]

In [48]:
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [49]:
history = model.fit(x_train,y_train, epochs=25,batch_size=32,validation_data = (x_test,y_test),callbacks=callbacks)

Epoch 1/25
248/248 [==============================] - 19s 72ms/step - loss: 541.9844 - rmse: 23.2806 - mae: 17.1751 - mape: 78.7861 - val_loss: 426.8297 - val_rmse: 20.6599 - val_mae: 15.2009 - val_mape: 73.5821
Epoch 2/25
248/248 [==============================] - 17s 70ms/step - loss: 458.8934 - rmse: 21.4218 - mae: 15.9161 - mape: 79.0035 - val_loss: 416.8371 - val_rmse: 20.4166 - val_mae: 15.4475 - val_mape: 78.4757
Epoch 3/25
248/248 [==============================] - 17s 70ms/step - loss: 455.5669 - rmse: 21.3440 - mae: 15.8514 - mape: 78.3210 - val_loss: 416.5031 - val_rmse: 20.4084 - val_mae: 15.5380 - val_mape: 79.4576
Epoch 4/25
248/248 [==============================] - 17s 70ms/step - loss: 451.0703 - rmse: 21.2384 - mae: 15.7993 - mape: 78.8523 - val_loss: 415.9895 - val_rmse: 20.3958 - val_mae: 15.4595 - val_mape: 78.5939
Epoch 5/25
248/248 [==============================] - 17s 70ms/step - loss: 446.1985 - rmse: 21.1234 - mae: 15.7519 - mape: 78.6940 - val_loss: 432.0810

In [50]:
cnn_pred=model.predict(test2)
cnn_pred

array([[34.46884 ],
       [34.37521 ],
       [35.005016],
       [34.408062],
       [35.189003],
       [34.433136],
       [34.336784],
       [34.45018 ]], dtype=float32)